<a href="https://colab.research.google.com/github/hmtrii/tirg/blob/main/Compute_recall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALL

In [1]:
!git clone https://github.com/hmtrii/tirg.git

Cloning into 'tirg'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 80 (delta 42), reused 64 (delta 26), pack-reused 0
Unpacking objects: 100% (80/80), done.


In [2]:
!rm -rf /content/Fashion200k
!mkdir /content/Fashion200k
!cp -r /content/drive/MyDrive/TIRG/dataset/fashion-200k/labels /content/Fashion200k
!cp /content/drive/MyDrive/TIRG/dataset/test_queries.txt /content/Fashion200k
!mkdir /content/Fashion200k/women

In [3]:
!tar -xf /content/drive/MyDrive/TIRG/dataset/fashion-200k/women.tar.gz -C /content/Fashion200k/women

In [4]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 17.2MB/s 


In [5]:
!pip install torch==1.2.0 torchvision==0.4.0    

     |████████████████████████████████| 748.9MB 23kB/s 
     |████████████████████████████████| 8.8MB 29.1MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [6]:
!pip install Pillow==5.2.0

     |████████████████████████████████| 2.0MB 24.7MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


# LOAD DATASET

In [1]:
cd /content/tirg

/content/tirg


In [2]:
class Opt:
    def __init__(self):
        self.dataset = "fashion200k"
        self.dataset_path = "/content/Fashion200k"
        self.batch_size = 32
        self.embed_dim = 512
        self.hashing = True
        self.retrieve_by_random = True
        self.model = "tirg"
        self.learning_rate = 1e-2
        self.weight_decay = 1e-6
        self.pretrained_model = "/content/drive/MyDrive/TIRG/checkpoint_fashion200k.pth"

In [ ]:
from test_retrieval import test
from main import load_dataset, create_model_and_optimizer

opt = Opt()
trainset, testset = load_dataset(opt)
model, _ = create_model_and_optimizer(opt, [t for t in trainset.get_all_texts()])
model.eval()

Compute all images feature

In [13]:
from tqdm import tqdm
import torch
import numpy as np

all_imgs = []
imgs = []
for i in tqdm(range(len(testset.imgs))):
    imgs += [testset.get_img(i)]
    if len(imgs) >= opt.batch_size or i == len(testset.imgs) - 1:
        if 'torch' not in str(type(imgs[0])):
            imgs = [torch.from_numpy(d).float() for d in imgs]
        imgs = torch.stack(imgs).float()

        if torch.cuda.is_available():
            imgs = torch.autograd.Variable(imgs).cuda()
        else:
            imgs = torch.autograd.Variable(imgs).cpu()

        imgs = model.extract_img_feature(imgs).data.cpu().numpy()
        all_imgs += [imgs]
        imgs = []
all_imgs = np.concatenate(all_imgs)
all_captions = [img['captions'][0] for img in testset.imgs]

100%|██████████| 29789/29789 [03:47<00:00, 131.22it/s]


Compute all queries feature

In [14]:
imgs = []
mods = []
all_queries = []
test_queries = testset.get_test_queries()
for t in tqdm(test_queries):
    imgs += [testset.get_img(t['source_img_id'])]
    mods += [t['mod']['str']]
    if len(imgs) >= opt.batch_size or t is test_queries[-1]:
        if 'torch' not in str(type(imgs[0])):
            imgs = [torch.from_numpy(d).float() for d in imgs]
        imgs = torch.stack(imgs).float()

        if torch.cuda.is_available():
            imgs = torch.autograd.Variable(imgs).cuda()
        else:
            imgs = torch.autograd.Variable(imgs).cpu()

        mods = [t for t in mods]
        f = model.compose_img_text(imgs, mods).data.cpu().numpy()
        all_queries += [f]
        imgs = []
        mods = []
all_queries = np.concatenate(all_queries)
all_target_captions = [t['target_caption'] for t in test_queries]

  0%|          | 14/33480 [00:00<04:01, 138.80it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 33480/33480 [04:16<00:00, 130.39it/s]


In [15]:
for i in range(all_queries.shape[0]):
    all_queries[i, :] /= np.linalg.norm(all_queries[i, :])
for i in range(all_imgs.shape[0]):
    all_imgs[i, :] /= np.linalg.norm(all_imgs[i, :])

In [30]:
# Create random queries
id_random = np.random.choice(all_queries.shape[0], 3000)
queries_feature = []
for id in id_random:
    queries_feature += [all_queries[id]]

queries_feature = np.array(queries_feature)

# Normal

In [31]:
import time

tic = time.time()
sims = queries_feature.dot(all_imgs.T)
for i, t in enumerate(test_queries):
	try:
		sims[i, t["source_img_id"]] = -10e10  # remove query image
	except:
		pass

nn_result = [np.argsort(-sims[i, :])[:110] for i in range(sims.shape[0])]
out = []
nn_result = [[all_captions[nn] for nn in nns] for nns in nn_result]
for k in [1, 5, 10, 50, 100]:
    r = 0.0
    # for i, nns in enumerate(nn_result):
    for id, nns in zip(id_random, nn_result):
        if all_target_captions[id] in nns[:k]:
            r += 1
    r /= len(nn_result)
    out += [('recall_top' + str(k) + '_correct_composition', r)]
tac = time.time()
print(out)
print(f"{round(tac - tic, 3)}s")

[('recall_top1_correct_composition', 0.025), ('recall_top5_correct_composition', 0.08766666666666667), ('recall_top10_correct_composition', 0.12533333333333332), ('recall_top50_correct_composition', 0.3006666666666667), ('recall_top100_correct_composition', 0.39566666666666667)]
8.238s


# Hashing

In [18]:
from LSH import create_hash_table

def compute_similary_by_hasing(all_imgs_feature, queries_feature, test_queries):
	hash_table = create_hash_table(all_imgs_feature, 10, 512)
	sims = []
	for i, query in enumerate(queries_feature):
		idx_imgs = hash_table.__getitem__(query)
		spatial_search = np.array([all_imgs_feature[i] for i in idx_imgs])
		if spatial_search.shape[0] != 0:
			dis_value = query.dot(spatial_search.T)
			sim = np.full(all_imgs_feature.shape[0], -10e10)
			for i in idx_imgs:
				sim[i] = dis_value[0]
				dis_value = dis_value[1:]
		else:
			sim = query.dot(all_imgs_feature.T)

		sim[test_queries[i]['source_img_id']] = -10e10
		sims += [sim]

	return np.array(sims)

In [35]:
tic = time.time()
sims = compute_similary_by_hasing(all_imgs, queries_feature, test_queries)
nn_result = [np.argsort(-sims[i, :])[:100] for i in range(sims.shape[0])]
out = []
nn_result = [[all_captions[nn] for nn in nns] for nns in nn_result]
for k in [1, 5, 10, 50, 100]:
    r = 0.0
    # for i, nns in enumerate(nn_result):
    for id, nns in zip(id_random, nn_result):
        if all_target_captions[id] in nns[:k]:
            r += 1
    r /= len(nn_result)
    out += [('recall_top' + str(k) + '_correct_composition', r)]
tac = time.time()
print(out)
print(f"{round(tac - tic, 3)}s")

[('recall_top1_correct_composition', 0.028), ('recall_top5_correct_composition', 0.06733333333333333), ('recall_top10_correct_composition', 0.10233333333333333), ('recall_top50_correct_composition', 0.215), ('recall_top100_correct_composition', 0.25166666666666665)]
5.535s
